# Problemas

Como vimos acima, há muitos passos na criação e definição de uma nova rede neural.
A grande parte desses ajustes dependem diretamente do problemas.

Abaixo, listamos alguns problemas. Todos os problemas e datasets usados vem do [Center for Machine Learning and Intelligent Systems](http://archive.ics.uci.edu/ml/datasets.php).


**Seu objetivo é determinar e implementar um modelo para cada problema.**

Isso inclui:

1. definir uma arquitetura.
Por enquanto usando somente camadas [Lineares](https://pytorch.org/docs/stable/nn.html#linear), porém podemos variar as ativações, como [Sigmoid](https://pytorch.org/docs/stable/nn.html#sigmoid), [Tanh](https://pytorch.org/docs/stable/nn.html#tanh), [ReLU](https://pytorch.org/docs/stable/nn.html#relu), [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html), [ELU](https://pytorch.org/docs/stable/generated/torch.nn.ELU.html), [SeLU](https://pytorch.org/docs/stable/generated/torch.nn.SELU.html), [PReLU](https://pytorch.org/docs/stable/generated/torch.nn.PReLU.html), [RReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html)
2. definir uma função de custo. Algums opções que vimos previamente incluem[L1](https://pytorch.org/docs/stable/nn.html#l1loss), [L2/MSE](https://pytorch.org/docs/stable/nn.html#mseloss), [Huber/SmoothL1](https://pytorch.org/docs/stable/nn.html#smoothl1loss), [*Cross-Entropy*](https://pytorch.org/docs/stable/nn.html#crossentropyloss), [Hinge](https://pytorch.org/docs/stable/nn.html#hingeembeddingloss)), e
3. definir um algoritmo de otimização ([SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [RMSProp](https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop), [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam))

A leitura do dado assim como a função de treinamento já estão implementados para você.

# Preâmbulo

In [1]:
# imports basicos
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import os
import sys
import time
import numpy as np

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

device(type='cuda')

## Funções básicas

Use a função `load_array ` declarada a seguir se voce ja tem os dados armazenados em um **array** (por exemplo um array do numpy, o `np.array`). Pode acontecer de que os nossos dados vêm simplesmente de um dataset que pode ser armazennado em um array, e portanto não é necessário fazer os outros passos mais complicados como carregar os dados do disco, etc; basta que possamos recuperar esses dados em *batches* aleatórios. O resultado dessa função é um `DataLoader` do Pytorch com os dados que fornecemos de entrada, e que permite que os acessamos da seguinte forma:

```python
data_loader = load_array(X, y, batch_size=32, is_train=True)
for x_batch, y_batch in data_loader:
    ### ... nossa iteração de treinamento aqui.
```

Essa função recebe como parâmetro os seguintes valores:

- `features`: um array que contém as features de todas as instâncias do dataset. Por exemplo, no caso do MNIST seria um array de tamanho `(60000, 28, 28, 1)` com todas as imagens do dataset de treino.
- `labels`: um array que contém os rótulos de cada instância de dados. No caso do MNIST, seria um array de tamanho `(60000,)` em que a posição `i` contém o rótulo do dígito da posição `i` do array `features`.
- `batch_size`: tamanho do batch desejado
- `is_train`: um booleano que indica se o dataset que estamos criando é o conjunto de treinamento ou não (conjunto de teste). A única mudança que isso causa no `Dataloader` resultante é que se for o conjunto de treinamento ele cria batches aleatórios.


In [4]:
def load_array(features, labels, batch_size, is_train=True):
    """Construct a Torch data loader"""

    ## transform the input arrays in a tensor in case they are not
    if type(features) != torch.tensor:
        features = torch.tensor(features)
    if type(labels) != torch.tensor:
        labels = torch.tensor(labels)

    ## create a Pytorch Dataset and DataLoader with the input data
    dataset = torch.utils.data.TensorDataset(features, labels)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)

Use a função `evaluate_accuracy` para calcular a acurácia e a *loss-function* para a rede em um conjunto de dados. Note que essa função pode ser usada tanto para avaliar a rede no conjunto de teste (no caso que usamos o `DataLoader` de teste) quanto o conjunto de treinamento (se usamos o `DataLoader` de treinamento). Os parâmetros são:
- `data_iter`: um `DataLoader` que contém os dados que queremos usar para avaliar a rede. Repare que esse parâmetro tipicamente é o ojeto que obtemos como saída da função `load_array` para montar o nosso `DataLoader`.
- `net`: a rede que queremos avaliar
- `loss`: a nossa *loss-function*. Pode ser um objeto de qualquer uma das funções de perda que mencionamos acima no começo do notebook.

O resultado dessa função é uma tupla em que o primeiro valor é a acurácia e o segundo a função de custo calculados.

In [5]:
# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    ## valores "acumuladores", que guardam a soma de, respectivamente, quantas instâncias
    ## prevemos corretamente, quantas instâncias percorremos no dataset, e o valor da loss; para
    ## todos os batches
    acc_sum, n, l = 0, 0, 0

    ## muda a rede para o "modo de teste". O que isso faz é mudar o comportamento de alguns módulos da rede,
    ## como os módulos de Dropout e BatchNorm, que funcionam de forma diferente quando estamos treinando ou
    ## quando estamos avaliando (ou usando em produção) a rede
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y.long())

          ## aqui estamos calculando a quantidade de previsões que temos correta para o batch atual. o resultado
          ## do argmax é a posição de `y_hat` que possui o maior valor. Consequentemente isso resulta na classe que
          ## a rede deu o maior score.
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()

          ##
          n += y.size(0)

    return acc_sum / n, l.item() / len(data_iter)

A função `train_validate` é a função que implementa nossas iterações de treinamento padrão. Ela ja faz o trabalho de percorrer o dataset inteiro para cada época, e também de tempos em tempos avaliar a rede e mostar os resultados na tela. Para isso ela faz chamadas à função `evaluate_accuracy` declarada anteriormente (entre outras coisas). Essa função tem os segugintes parâmetros:
- `net`: a rede que queremos treinar
- `train_iter` e `test_iter`: nossos `DataLoaders` que criamos para acessar os dados. Esses DataLoaders podem ser criados com a função `load_array` declarada acima.
- `trainer`: é o nosso otimizador. Podemos usar aqui qualquer um dos otimizadores que escolhermos da lista citada no começo desse notebook.
- `loss`: a loss function que escolhemos para otimizar. Pode ser qualquer um das funções de custo citadas no começo do notebook.
- `num_epochs`: a quantidade de épocas pelas quais queremos que o treinamento ocorra.
- `type`: o tipo de tarefa que estamos lidando. Se for um problema de regressão, usamos `type='regression'`, e se for um problema de classificação, usamos `type='classification'`. Esse parâmetro é necessário para a função, por exemplo, saber quais métricas ele vai mostrar (acurácia, ou apenas o MSE, etc.)

In [6]:
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, trainer, loss, num_epochs, type='regression'):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            if type == 'regression':
              l = loss(y_hat, y.float())
            else:
              l = loss(y_hat, y.long())
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size(0)
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        if type == 'regression':
          print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                % (epoch + 1, train_l_sum / len(train_iter), test_loss, time.time() - start))
        else:
          print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))


Use a função a seguir para inicializar os pesos da rede. Ela recebe como parâmetro um módulo da rede neural, e se for uma camada linear ele inicializa os pesos e os bias dessa camada. Embora possa parecer complicado de precisar chamar essa função para todas as camadas lineares da nossa rede, o módulos do Pytorch (que incluem tanto as redes criadas com o `nn.Sequential` ou com `nn.Module`) possuem a função `net.apply()` que recebe como parâmetro uma função e aplica ela a todos os submódulos da rede. Portanto, depois de ter criado a nossa rede, podemos chamar:

```python
net.apply(weights_init)
```
que automaticamente todas as camadas `nn.Linear` serão inicializadas. Caso queira saber mais sobre o `.apply()`, veja o seguinte [link](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.apply).

In [7]:
# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

# Problema 1

Neste problema, você receberá 14 *features* coletadas de pacientes e tentará predizer se eles tem algum sinal de doença cardíaca. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

In [8]:
## aqui fazemos o download do dataset usando o `!wget`. Se estamos rodando em um servidor linux (como é o caso do Colab),
## podemos usar comandos do linux precedidos pelo "!". Por exemplo podemos fazer !ls para listar os arquivos da instância do colab.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

## aqui fazemos um tratamento inicial dos dados. "np.genfromtxt" lê os dados de um arquivo .txt e transforma em
## um array. Pode ser interessante abrir o arquivo para verificar como os dados chegaram. Se estiver no colab, voce
## pode verificar o arquivo "processed.cleveland.data" clicando na pastinha do canto esquerdo da página. a função
## "np.nan_to_num" trata valores NaN e infinitos no dataset.
data = np.genfromtxt('processed.cleveland.data', delimiter=',', dtype=np.float32)
data = np.nan_to_num(data)

--2025-05-19 12:37:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘processed.cleveland.data’

processed.cleveland     [ <=>                ]  18.03K   109KB/s    in 0.2s    

2025-05-19 12:37:42 (109 KB/s) - ‘processed.cleveland.data’ saved [18461]



In [9]:
## aqui separamos os dados entre features (X) e rótulo (y), e depois separamos em um conjunto de treinamento e teste
print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.20, random_state=42)

(303, 14) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6.    0. ]
(303, 13) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6. ]
(303,) 0.0


In [10]:
## aqui criamos nossos DataLoaders para conseguirmos iterar nos dados
batch_size = 64
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [11]:
# 1) Converta labels para binário (0 ou 1)
train_labels_bin = (train_labels > 0).astype(np.int64)
test_labels_bin  = (test_labels  > 0).astype(np.int64)

# 2) Crie novos DataLoaders usando load_array com rótulos binários
batch_size = 64  # já estava definido acima
train_iter = load_array(train_features, train_labels_bin, batch_size, is_train=True)
test_iter  = load_array(test_features,  test_labels_bin,  batch_size, is_train=False)

# 3) Definição da rede (MLP simples: 14 → 64 → 32 → 2)
# Substitua “14” por train_features.shape[1], que deve ser 13
in_dim = train_features.shape[1]  # normalmente 13
net = nn.Sequential(
    nn.Linear(in_dim, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)

net.apply(weights_init)
net.to(device)

# 4) Loss e otimizador
loss_fn = nn.CrossEntropyLoss()
trainer = optim.Adam(net.parameters(), lr=0.001)

# 5) Treinamento e validação
#    (observe que aqui passamos type='classification')
train_validate(net, train_iter, test_iter, trainer, loss_fn, num_epochs=20, type='classification')

# 6) Mostre as 5 primeiras previsões no conjunto de teste
X5 = torch.tensor(test_features[0:5], dtype=torch.float32).to(device)
logits = net(X5)
preds = logits.argmax(dim=1).cpu().numpy()
print("Predições:", preds)
print("Verdadeiros (binário):", test_labels_bin[0:5])


training on cuda
epoch 1, train loss 0.6937, train acc 0.496, test loss 0.6925, test acc 0.475, time 0.7 sec
epoch 2, train loss 0.6909, train acc 0.558, test loss 0.6940, test acc 0.475, time 0.0 sec
epoch 3, train loss 0.6862, train acc 0.558, test loss 0.6954, test acc 0.475, time 0.0 sec
epoch 4, train loss 0.6860, train acc 0.558, test loss 0.7004, test acc 0.475, time 0.0 sec
epoch 5, train loss 0.6820, train acc 0.558, test loss 0.6948, test acc 0.475, time 0.0 sec
epoch 6, train loss 0.6783, train acc 0.558, test loss 0.6835, test acc 0.475, time 0.0 sec
epoch 7, train loss 0.6713, train acc 0.562, test loss 0.6771, test acc 0.492, time 0.0 sec
epoch 8, train loss 0.6646, train acc 0.583, test loss 0.6681, test acc 0.557, time 0.0 sec
epoch 9, train loss 0.6497, train acc 0.587, test loss 0.6586, test acc 0.607, time 0.0 sec
epoch 10, train loss 0.6416, train acc 0.624, test loss 0.6448, test acc 0.656, time 0.0 sec
epoch 11, train loss 0.6300, train acc 0.645, test loss 0.6345

# Problema 2

Neste problema, você receberá 90 *features* extraídas de diversas músicas (datadas de 1922 até 2011) e deve predizer o ano de cada música. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD

In [12]:
# download do dataset
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
!unzip YearPredictionMSD.txt.zip
data = np.genfromtxt('YearPredictionMSD.txt', delimiter=',', dtype=np.float32)

print(data[0, :])
X, y = data[:, 1:], data[:, 0]
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-19 12:37:59--  http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘YearPredictionMSD.txt.zip’

YearPredictionMSD.t     [  <=>               ] 201.24M  8.24MB/s    in 21s     

2025-05-19 12:38:21 (9.49 MB/s) - ‘YearPredictionMSD.txt.zip’ saved [211011981]

Archive:  YearPredictionMSD.txt.zip
  inflating: YearPredictionMSD.txt   
[ 2.0010000e+03  4.9943569e+01  2.1471140e+01  7.3077499e+01
  8.7486095e+00 -1.7406281e+01 -1.3099050e+01 -2.5012020e+01
 -1.2232570e+01  7.8308902e+00 -2.4678299e+00  3.3213601e+00
 -2.3152101e+00  1.0205560e+01  6.1110913e+02  9.5108960e+02
  6.9811426e+02  4.0898486e+02  3.8370911e+02  3.2651511e+02
  2.3811327e+02  2.5142413e+02  1.8717351e+02  1.0042652e+02
  1.7919498e+02 -8

In [13]:
# 1) Definição da rede (90 → 128 → 64 → 1)
net = nn.Sequential(
    nn.Linear(90, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)
net.apply(weights_init)
net.to(device)

# 2) Loss e otimizador (MSE para regressão)
loss_fn = nn.MSELoss()
trainer = optim.Adam(net.parameters(), lr=0.001)

# 3) Treinamento e validação
#    (note type='regression')
train_validate(net, train_iter, test_iter, trainer, loss_fn, num_epochs=10, type='regression')

# 4) Mostre as 5 primeiras previsões no conjunto de teste
X5 = torch.tensor(test_features[0:5, :], dtype=torch.float32).to(device)
preds = net(X5).detach().cpu().numpy().reshape(-1)
print("Previsões:", preds)
print("Verdadeiros:", test_labels[0:5])


training on cuda


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([81])) that is different to the input size (torch.Size([81, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(inpu

epoch 1, train loss 111923.4511, test loss 21915.5650, time 10.1 sec
epoch 2, train loss 20181.5152, test loss 18164.2387, time 10.0 sec
epoch 3, train loss 17190.5193, test loss 20780.6655, time 10.2 sec
epoch 4, train loss 15910.3928, test loss 15040.0894, time 10.3 sec
epoch 5, train loss 14977.2437, test loss 14438.9101, time 10.2 sec
epoch 6, train loss 13849.7679, test loss 16514.6373, time 10.3 sec
epoch 7, train loss 13385.4365, test loss 11951.7484, time 9.6 sec
epoch 8, train loss 12667.7801, test loss 13426.1999, time 10.2 sec
epoch 9, train loss 12151.1241, test loss 11337.6696, time 10.2 sec
epoch 10, train loss 11812.7895, test loss 11021.0888, time 10.2 sec
Previsões: [2048.9194 2099.081  2171.239  1719.4153 1912.717 ]
Verdadeiros: [2008. 2001. 2006. 2008. 1998.]


In [14]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = net(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])

tensor([[2048.9194],
        [2099.0811],
        [2171.2390],
        [1719.4153],
        [1912.7170]], device='cuda:0', grad_fn=<AddmmBackward0>) [2008. 2001. 2006. 2008. 1998.]


# Problema 3

Neste problema, você receberá várias *features* (como altura média, inclinação, etc) descrevendo uma região e o modelo deve predizer qual o tipo da região (floresta, montanha, etc). Mais informações sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/covertype

In [15]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip covtype.data.gz
data = np.genfromtxt('covtype.data', delimiter=',', dtype=np.float32)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)
train_labels = train_labels - 1
test_labels = test_labels - 1

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-19 12:44:01--  http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘covtype.data.gz’

covtype.data.gz         [         <=>        ]  10.72M  3.99MB/s    in 2.7s    

2025-05-19 12:44:04 (3.99 MB/s) - ‘covtype.data.gz’ saved [11240707]

gzip: covtype.data.gz already has .gz suffix -- unchanged
(581012, 55) [2.596e+03 5.100e+01 3.000e+00 2.580e+02 0.000e+00 5.100e+02 2.210e+02
 2.320e+02 1.480e+02 6.279e+03 1.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 

In [16]:
# 1) Definição da rede (54 → 128 → 64 → 7)
net = nn.Sequential(
    nn.Linear(54, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 7)
)
net.apply(weights_init)
net.to(device)

# 2) Loss e otimizador
loss_fn = nn.CrossEntropyLoss()
trainer = optim.Adam(net.parameters(), lr=0.001)

# 3) Treinamento e validação (classificação)
train_validate(net, train_iter, test_iter, trainer, loss_fn, num_epochs=20, type='classification')

# 4) Mostre as 5 primeiras previsões no conjunto de teste
X5 = torch.tensor(test_features[0:5, :], dtype=torch.float32).to(device)
preds = net(X5).argmax(dim=1).cpu().numpy()
print("Predições:", preds)
print("Verdadeiros:", test_labels[0:5])


training on cuda
epoch 1, train loss 0.7442, train acc 0.677, test loss 0.6518, test acc 0.717, time 11.0 sec
epoch 2, train loss 0.6483, train acc 0.717, test loss 0.6315, test acc 0.721, time 11.2 sec
epoch 3, train loss 0.6226, train acc 0.727, test loss 0.6023, test acc 0.736, time 11.2 sec
epoch 4, train loss 0.6066, train acc 0.735, test loss 0.5883, test acc 0.748, time 11.2 sec
epoch 5, train loss 0.5923, train acc 0.742, test loss 0.6189, test acc 0.731, time 11.2 sec
epoch 6, train loss 0.5795, train acc 0.748, test loss 0.5739, test acc 0.752, time 11.1 sec
epoch 7, train loss 0.5717, train acc 0.752, test loss 0.5512, test acc 0.760, time 11.2 sec
epoch 8, train loss 0.5636, train acc 0.755, test loss 0.5447, test acc 0.762, time 11.1 sec
epoch 9, train loss 0.5559, train acc 0.758, test loss 0.5699, test acc 0.753, time 11.3 sec
epoch 10, train loss 0.5491, train acc 0.762, test loss 0.5312, test acc 0.774, time 11.3 sec
epoch 11, train loss 0.5474, train acc 0.762, test l

# Problema 4

Neste problema, você receberá 11 *features* extraídas de tipos de vinhos, e terá que predizer um *score* para cada vinho. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [17]:
# download do dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
data_red = np.genfromtxt('winequality-red.csv', delimiter=';', dtype=np.float32, skip_header=1)
data_white = np.genfromtxt('winequality-white.csv', delimiter=';', dtype=np.float32, skip_header=1)
data = np.concatenate((data_red, data_white), axis=0)
data = np.nan_to_num(data)

print(data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-19 12:50:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-red.csv’

winequality-red.csv     [   <=>              ]  82.23K   166KB/s    in 0.5s    

2025-05-19 12:50:55 (166 KB/s) - ‘winequality-red.csv’ saved [84199]

--2025-05-19 12:50:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-white.csv’

winequality-white.c     [   <=>              ] 258.23K   392KB/s    in 0.7s    

2025-05-19

In [18]:
# 1) Definição da rede (11 → 64 → 32 → 1)
net = nn.Sequential(
    nn.Linear(11, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
net.apply(weights_init)
net.to(device)

# 2) Loss e otimizador (MSE para regressão)
loss_fn = nn.MSELoss()
trainer = optim.Adam(net.parameters(), lr=0.001)

# 3) Treinamento e validação (regressão)
train_validate(net, train_iter, test_iter, trainer, loss_fn, num_epochs=10, type='regression')

# 4) Mostre as 5 primeiras previsões no conjunto de teste
X5 = torch.tensor(test_features[0:5, :], dtype=torch.float32).to(device)
preds = net(X5).detach().cpu().numpy().reshape(-1)
print("Previsões:", preds)
print("Verdadeiros:", test_labels[0:5])


training on cuda
epoch 1, train loss 22.5988, test loss 7.4813, time 0.2 sec


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([52])) that is different to the input size (torch.Size([52, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([45])) that is different to the input size (torch.Size([45, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 2, train loss 5.3107, test loss 4.5352, time 0.2 sec
epoch 3, train loss 3.7196, test loss 3.0502, time 0.2 sec
epoch 4, train loss 2.1815, test loss 1.4511, time 0.2 sec
epoch 5, train loss 1.0770, test loss 0.8801, time 0.2 sec
epoch 6, train loss 0.8882, test loss 0.8599, time 0.2 sec
epoch 7, train loss 0.8827, test loss 0.8802, time 0.2 sec
epoch 8, train loss 0.8850, test loss 0.8543, time 0.2 sec
epoch 9, train loss 0.8783, test loss 0.8533, time 0.2 sec
epoch 10, train loss 0.8771, test loss 0.8460, time 0.2 sec
Previsões: [6.2013845 5.8325014 6.0860577 5.686197  5.9482117]
Verdadeiros: [8. 5. 7. 6. 6.]


In [19]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = net(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])

tensor([[6.2014],
        [5.8325],
        [6.0861],
        [5.6862],
        [5.9482]], device='cuda:0', grad_fn=<AddmmBackward0>) [8. 5. 7. 6. 6.]
